In [ ]:
%cd ..
%cd ..

In [2]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils

import Notebooks.Scripts.Useful as use
from Notebooks.Scripts.sampling_utils import get_forcing

In [ ]:
df_two = pd.read_csv('fitted_stats/2d_sims.csv')
event = df_two[df_two['P Mag [mm/hr]'] > 50].reset_index(drop = True)
event

In [4]:
model_folder = 'Models/Placeholder'
template_folder = Path('Models/FloodAdapt_stolen')
sfincs_model = SfincsModel(root=template_folder, mode='r')
sfincs_model.read()
s_mag, p_mag, s_dur, p_dur, p_lag, t_mag, base_flow = get_forcing(event, 0)

In [ ]:
use.create_empty_folder(model_folder)

In [ ]:
sfincs_model.set_root(model_folder, mode = 'w+')
sfincs_model.write()
span = 4
peak_ind = use.retrieve_historical_tide(template_folder, sfincs_model, mag = t_mag, span = span)

tide = use.index_first(sfincs_model.forcing['bzs'].copy())
time_ind = tide[0].coords['time'].values

center = time_ind[peak_ind]

sfincs_model.forcing.pop("bzs", None)  # reset
surge_da, start_s, end_s = use.artificial_driver(time_ind, mag = s_mag, dur = int(12.42 * s_dur), center = center, # assume m2 tidal cycle when setting duration
                                                    name = r'Surge [$m$]', lag = 0, base = 0.2, cutoff = True, surge = True) # base is threhold used for duration in this case: should try and automate
wl_bound_surge = tide[:] + surge_da

# sf.write_forcing() only overwrites boundary points within domain
sfincs_model.setup_waterlevel_forcing(wl_bound_surge, 'sfincs.bnd')
sfincs_model.write_forcing() 

sfincs_model.forcing.pop("precip", None)  # reset
artificial_precip, start_p, end_p = use.artificial_driver(time_ind, mag = p_mag, dur = int(p_dur), center = center,
                                                                name = r'Precipitation [$mm$ $h^{-1}$]', lag = p_lag)

sfincs_model.setup_precip_forcing(artificial_precip.to_dataframe())
sfincs_model.write_forcing() 

sfincs_model.forcing.pop("dis", None)  # reset
artificial_discharge, start_d, end_d = use.artificial_driver(time_ind, mag = 0, dur = 0, center = center,
                                                                name = 1, lag = 0, base = base_flow)

dis_gdf = utils.read_xy(os.path.join(template_folder, "sfincs.src"), crs=sfincs_model.crs)
sfincs_model.setup_discharge_forcing(timeseries = artificial_discharge.to_dataframe(),
                                        locations = dis_gdf)


sfincs_model.write_forcing() 

real_start =  min(start_s, start_p, start_d).astype('datetime64[s]').astype('datetime64[us]').astype('O')
real_stop =  max(end_s, end_p, end_d).astype('datetime64[s]').astype('datetime64[us]').astype('O')


sfincs_model.setup_config(
    tref = "20200101 000000",
    tstart = real_start.strftime('%Y%m%d %H%M%S'),
    tstop = real_stop.strftime('%Y%m%d %H%M%S'),
)
sfincs_model.write()

In [7]:
center_p = start_p + (end_p - start_p) / 2

In [ ]:
fig, ax = plt.subplots(4, 1, figsize = ((8, 10)), gridspec_kw={'height_ratios': [1, 1, 1, 2]})
ax[0].axhline(0.2, c = 'b', lw = 0.5)
ax[0].set_ylabel('MSL [m+MSL]', fontsize = 12, color = 'b')
ax[0].yaxis.tick_right()
ax[0].tick_params(axis='y', colors='b')
ax[0].yaxis.set_label_position("right")
ax[0].set_yticks([0.2], labels = ['0.2'], fontsize = 12)
ax[0].set_ylim([0, 0.4])

for i in ['bottom', 'top', 'right', 'left']:
    ax[0].spines[i].set_visible(False)

tide.plot(ax = ax[1], c = 'b', lw = 0.5)
ax[1].set_title('')
ax[1].set_xlabel('')
ax[1].set_ylabel('Tide [m]', fontsize = 12, color = 'b')
ax[1].yaxis.tick_right()
ax[1].tick_params(axis='y', colors='b')
ax[1].yaxis.set_label_position("right")
ax[1].axhline(tide.sel({'time': center}).item(),
              (12*s_dur/2) / (12*s_dur), 1, c = 'b', ls = ':') # 12 hardcoded, average tidal duration
ax[1].set_yticks([0, tide.sel({'time': center}).item()], labels = ['0', 'T Mag'], fontsize = 12)

for i in ['bottom', 'top', 'right', 'left']:
    ax[1].spines[i].set_visible(False)


surge_da.plot(ax = ax[2], color = 'b', lw = 0.5)
ax[2].set_ylabel('Skew surge [m]', fontsize = 12, color = 'b')
ax[2].set_xlabel('')
ax[2].yaxis.tick_right()
ax[2].tick_params(axis='y', colors='b')
ax[2].yaxis.set_label_position("right")
ax[2].annotate('', xy=(start_s, 0.19), xytext=(end_s, 0.19),
               arrowprops=dict(arrowstyle='<->', color='b', lw=1))
ax[2].text(center, 0.2, 'S Dur', fontsize = 12,
           fontweight = 'bold', ha='center', va='bottom', color = 'b')
ax[2].axhline(surge_da.max().item(),
              (12*s_dur/2) / (12*s_dur), 1, c = 'b', ls = ':') # 12 hardcoded, average tidal duration
ax[2].set_yticks([0, 0.2, surge_da.max().item()], labels = ['0', '0.2', 'S Mag'], fontsize = 12)


for i in ['bottom', 'top', 'right', 'left']:
    ax[2].spines[i].set_visible(False)

wl_bound_surge.plot(ax = ax[3], c = 'b', lw = 2)
ax[3].set_title('')
ax[3].set_ylabel('Still water level [m+MSL]', fontsize = 12, color = 'b')
ax[3].yaxis.set_label_position("right")
ax[3].axhline(wl_bound_surge.sel({'time': center}).item(),
              (12*s_dur/2) / (12*s_dur), 1, c = 'b', ls = ':')
ax[3].set_yticks([0, wl_bound_surge.sel({'time': center}).item()], labels = ['0', 'S Mag + T Mag + 0.2'], fontsize = 12)
new_ax = ax[3].twinx()
new_ax.yaxis.tick_left()
new_ax.yaxis.set_label_position("left")
ax[3].yaxis.tick_right()
ax[3].tick_params(axis='y', colors='b')
artificial_precip.plot(ax = new_ax, color = 'k', lw = 2)
new_ax.annotate('', xy=(start_p, 0), xytext=(end_p, 0),
               arrowprops=dict(arrowstyle='<->', color='k', lw=1))
new_ax.text(center_p, 2, 'P Dur', fontsize = 12,
            fontweight = 'bold', ha='center', va='center')
new_ax.axhline(artificial_precip.max().item(),
               0, (12*(s_dur+1)/2 + p_lag)/ (12*(s_dur+1)), c = 'k', ls = ':') # 12 hardcoded, average tidal duration
new_ax.set_yticks([0, artificial_precip.max().item()], labels = ['0', 'P Mag'], fontsize = 12)
new_ax.set_ylabel(r'Precipitation [mm h$^{-1}$]', fontsize = 12)

new_ticks = [center - np.timedelta64(24, 'h'), center_p, center,
             center + np.timedelta64(24, 'h')]

ax[3].set_xticks(new_ticks,
                labels = ['-24', 'P Lag', '0', '24'],
                color = 'r',
                fontsize = 12)
for i in range(3):
    ax[i].set_xticks(new_ticks[0:2] + [new_ticks[-1]],
                    labels = ['', '', ''],
                    color = 'r',
                    fontsize = 12)
for i in range(4):
    ax[i].tick_params(axis='x', colors='r')
    ax[i].set_xlim([start_s - np.timedelta64(6, 'h'), end_s + np.timedelta64(6, 'h')])

ax[3].set_xlabel('Relative time to peak in skew surge [h]', color = 'r', fontsize = 12)
ax[3].spines['top'].set_visible(False)
new_ax.spines['top'].set_visible(False)
ax[3].spines['right'].set_color('b')
new_ax.spines['right'].set_color('b')
new_ax.spines['bottom'].set_color('r')

ax[0].axvline(x=center,ymin=-1.1,ymax=1,c="red",linewidth=0.4,zorder=0, clip_on=False)
ax[1].axvline(x=center,ymin=-1.1,ymax=1.1,c="red",linewidth=0.4,zorder=0, clip_on=False)
ax[2].axvline(x=center,ymin=-1.1,ymax=1.1,c="red",linewidth=0.4,zorder=0, clip_on=False)
ax[3].axvline(x=center,ymin=0,ymax=1.1,c="red",linewidth=0.4,zorder=0, clip_on=False)
ax[3].axvline(x=center_p,ymin=0,ymax=50/52,c="red",linewidth=0.4,zorder=0)

# Add the plus symbol between the subplots
fig.text(0.5134, 0.735, '+', ha='center', va='center', fontsize=30, color='b', zorder = 2)
fig.text(0.5134, 0.565, '+', ha='center', va='center', fontsize=30, color='b', zorder = 2)
fig.text(0.5134, 0.4, '=', ha='center', va='center', fontsize=30, color='b', zorder = 2)

tick_labels = ax[1].get_yticklabels()
for label in tick_labels:
    if label.get_text() == 'T Mag':  # Specify the tick label to make bold
        label.set_fontweight('bold')
ax[1].set_yticklabels(tick_labels)

tick_labels = ax[2].get_yticklabels()
for label in tick_labels:
    if label.get_text() == 'S Mag':  # Specify the tick label to make bold
        label.set_fontweight('bold')
ax[2].set_yticklabels(tick_labels)

tick_labels = new_ax.get_yticklabels()
for label in tick_labels:
    if label.get_text() == 'P Mag':  # Specify the tick label to make bold
        label.set_fontweight('bold')
new_ax.set_yticklabels(tick_labels)

tick_labels = ax[3].get_xticklabels()
for label in tick_labels:
    if label.get_text() == 'P Lag':  # Specify the tick label to make bold
        label.set_fontweight('bold')
ax[3].set_xticklabels(tick_labels)

plt.savefig('Figures/PDF/f06.pdf', dpi=300, format='pdf', bbox_inches="tight")
plt.savefig('Figures/PNG/f06.png', format='png', bbox_inches="tight")

plt.show()